<a href="https://colab.research.google.com/github/TheRealMatri/SEO-Testful/blob/main/Testful.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries for Hugging Face Transformers, Datasets, JAX, and JAXLib
!pip install transformers datasets jax jaxlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
# Check installed version of Hugging Face transformers
!pip show transformers

Name: transformers
Version: 4.51.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [2]:
# If needed, upgrade transformers to the latest version
!pip install --upgrade transformers datasets jax jaxlib

In [4]:
# Import necessary libraries for model training
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset


/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.6.0, so it will not be used.
  warnings.warn(


In [8]:
# Load the pre-trained GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/rugpt3small_based_on_gpt2')
model = GPT2LMHeadModel.from_pretrained('ai-forever/rugpt3small_based_on_gpt2', torch_dtype=torch.float32)

In [18]:
# Load the train dataset (with 'train' split)
train_dataset = load_dataset('json', data_files='/content/dataset_train.jsonl', split='train')

# Load the test dataset (with 'test' split, but no split argument needed)
test_dataset = load_dataset('json', data_files='/content/dataset_test.jsonl', split='train')  # Use 'train' here as split name


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [19]:
# Define a label mapping with all possible categories
label_map = {
    "Дети": 0,
    "Локализация": 1,
    "Эстетика": 2,
    "Акции": 3,
    "Пенсионеры": 4,
    "Технологии": 5,
    "Ортодонтия": 6,
    "Экстренная помощь": 7,
}

# Function to encode labels
def encode_labels(example):
    example['label'] = label_map.get(example['label'], -1)  # Default to -1 if label is unknown
    return example

# Apply label encoding to both datasets
train_dataset = train_dataset.map(encode_labels)
test_dataset = test_dataset.map(encode_labels)

# Check the updated datasets (optional print)
print(train_dataset[0])  # Print the first item in the train dataset
print(test_dataset[0])   # Print the first item in the test dataset


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

{'prompt': 'Сделайте SEO-текст для стоматологии "DentCare". Услуга: лечение кариеса у детей. Уникальное условие: безболезненно.', 'completion': 'В "DentCare" заботливо лечим кариес у детей без боли. Дети довольны, родители спокойны!', 'label': 0}
{'prompt': 'Создайте SEO-объявление для стоматологии "DentCare". Услуга: отбеливание зубов. Уникальное условие: результат за одно посещение.', 'completion': 'Отбеливание в "DentCare" — сияющая улыбка всего за одно посещение!', 'label': 2}


In [20]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load model & tokenizer from your saved folder
model = GPT2LMHeadModel.from_pretrained('./fine_tuned_model')
tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_model')


In [21]:
# Define a function to tokenize the text
def tokenize_function(examples):
    # Concatenate the prompt and completion with a separator (e.g., a special token or just a space)
    inputs = [prompt + "\n" + completion for prompt, completion in zip(examples['prompt'], examples['completion'])]

    # Tokenize the inputs and ensure the correct format for labels
    model_inputs = tokenizer(inputs, truncation=True, padding=True, return_tensors="pt")

    # Ensure labels are shifted by one token for causal language modeling
    model_inputs['labels'] = model_inputs.input_ids.detach().clone()
    return model_inputs

# Apply the tokenization to the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

In [22]:
# Define training arguments optimized for GPU (14GB VRAM, 12GB RAM)
training_args = TrainingArguments(
    output_dir='./results',                # Output directory to save model and checkpoints
    logging_dir='./logs',                  # Directory for storing logs
    logging_steps=500,                     # Log every 500 steps
    save_steps=5000,                       # Save checkpoints every 5000 steps
    save_total_limit=3,                    # Only keep the last 3 checkpoints
    per_device_train_batch_size=48,         # Set batch size for training based on GPU RAM
    per_device_eval_batch_size=48,          # Set batch size for evaluation based on GPU RAM         # Accumulate gradients over 4 steps to reduce memory usage
    weight_decay=0.01,                     # Strength of weight decay
    num_train_epochs=3,                    # Number of epochs
    warmup_steps=500,                      # Number of warmup steps for learning rate scheduler
    logging_first_step=True,               # Log the first step of training
    load_best_model_at_end=True,           # Load the best model at the end of training
    metric_for_best_model='loss',     # Metric for selecting the best model
    save_strategy="steps",                 # Save the model every certain number of steps
    eval_strategy="steps",                 # Evaluate every certain number of steps
    eval_steps=200,                        # Evaluate every 200 steps
    dataloader_num_workers=2,              # Number of workers for loading data
    fp16=True,                             # Enable mixed precision training for faster training
    learning_rate=5e-5,                    # Set the learning rate
    report_to="wandb",                     # Log metrics to W&B (optional, remove if not needed)
)

In [23]:
# Create the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


In [ ]:
import time
from threading import Thread
from IPython.display import display, clear_output

# Set your desired training time in seconds (e.g., 1 hour)
TRAINING_TIME_LIMIT = 60*10  # 1 hour

# Flag to control training
stop_training = False

# Timer function
def stop_training_after_delay():
    global stop_training
    time.sleep(TRAINING_TIME_LIMIT)
    stop_training = True
    clear_output()
    print(f"⏰ Time's up! Stopping training after {TRAINING_TIME_LIMIT // 60} minutes.")

# Start timer
timer_thread = Thread(target=stop_training_after_delay)
timer_thread.start()

# Begin training loop
while not stop_training:
    trainer.train(resume_from_checkpoint=True)
    trainer.save_model("./fine_tuned_model")

print("✅ Training session ended.")


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Step,Training Loss,Validation Loss
200,7.811800,0.000425
400,7.811800,0.000055


In [15]:
# Evaluate the model on the test dataset
eval_results = trainer.evaluate()

# Print the evaluation results
print(eval_results)

{'eval_loss': 6.9561567306518555, 'eval_runtime': 0.2584, 'eval_samples_per_second': 154.815, 'eval_steps_per_second': 3.87, 'epoch': 3.0}


In [16]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json')

In [17]:
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model and tokenizer
model_path = "./fine_tuned_model"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Create a text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Input your prompt here
prompt = "Сгенерируй рекламу для стамоталогии Smile предлагающей очистку зубов со скидкой 20%"

# Generate output
output = generator(prompt, max_length=100, do_sample=True, top_k=50, top_p=0.95, temperature=0.8)

# Print generated text
print(output[0]['generated_text'])


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Сгенерируй рекламу для стамоталогии Smile предлагающей очистку зубов со скидкой 20%
&quot;Угнетающая реклама - это эффективный способ избавиться от угнетающего эффекта зубной пасты и других средств. В этом помогут наши услуги по очистке зубов.&quot;





21073468	mos-jkh	2019-12-05 16:00:00	Мои твиты   Сб, 13:14:
